In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import os

# Important Variables

In [3]:
image_size = (512, 512)
batch_size = 16
input_shape = image_size + (3,)
learning_rate = 1e-4
seed = 1337
validation_split = 0.2
epochs = 10

base_model = tf.keras.applications.ResNet50V2(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=5,
)

In [ ]:
from data_loader import load_data

train_ds, val_ds = load_data(image_size, batch_size, seed=seed, validation_split=validation_split)

In [ ]:
import tensorflow as tf

head_model = base_model.output
head_model = tf.keras.layers.GlobalAveragePooling2D()(head_model)
head_model = tf.keras.layers.Dense(5, activation="softmax")(head_model)
model = tf.keras.Model(inputs=base_model.input, outputs=head_model)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

# Model Fitting
The output of this is saved as 'runtime.txt'

In [ ]:
import datetime
import io
import os
import sys
import shutil
from tensorflow.keras.callbacks import Callback


model_id = base_model.name + '_' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = f"../records/{model_id}/logs"

array = []
class FinalEpochLogger(Callback):
    def __init__(self):
        super(FinalEpochLogger, self).__init__()

    def on_epoch_end(self, epoch, logs=None):
        # Store the output of the final epoch
        array.append(f"Epoch {epoch+1}: {logs}\n") # CLARIISE IM SO SORRY APPEND PALA SIYAA HINDI PUSH

final_epoch_logger = FinalEpochLogger()
history = model.fit(train_ds, epochs=epochs, validation_data=val_ds, callbacks=[
    tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1),
    final_epoch_logger
                                                                                ])

# Save the model
os.makedirs(f"/mnt/c/Users/Ann Clarisse Salazar/Documents/project/records/{model_id}", exist_ok=True)
model.save(f'/mnt/c/Users/Ann Clarisse Salazar/Documents/project/records/{model_id}/model_{model_id}.keras')

# Experiment Summary
This outputs details of the run to 'info.txt'. This includes the summary of the model's architecture, number of epochs, learning rate, etc.

If notes about this experiment are needed, feel free to add them to the corresponding cell.

In [9]:
note = ""

In [10]:
import io
import sys

output_capture = io.StringIO()

sys.stdout = output_capture

print(note + "\n\n")

# print training info
for element in array:
    print(element)

print("\n\n")

# print model info
print("HYPERPARAMETERS")
print(f"Model Name: {base_model.name}")
print(f"Epochs: {epochs}")
print(f"Batch Size: {batch_size}")
print(f"Image Size: {image_size}")
print(f"Seed: {seed}")
print(f"Validation Split: {validation_split}")
print(f"Learning Rate: {learning_rate}")

# print model architecture information
print("\n\nSUMMARY")
model.summary()

# Reset stdout to its original value
sys.stdout = sys.__stdout__

# Get the captured output
captured_output = output_capture.getvalue()


# Define the file path
file_path = f'/mnt/c/Users/Ann Clarisse Salazar/Documents/project/records/{model_id}/info.txt'

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# save to info.txt
with open(f"/mnt/c/Users/Ann Clarisse Salazar/Documents/project/records/{model_id}/info.txt", 'w') as file:
    file.write(captured_output)